# CLI

> This is the demo_garden cli

In [ ]:
#| default_exp cli

In [ ]:
#| export
from fastcore.script import *
import os
from pathlib import Path
from fh_bootstrap import *

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export

def generate_sample_requirements_txt():
    """Generates sample requirements.txt"""
    return """
python-fasthtml
"""

def generate_sample_readme_content():
    """Generates boiler plate README.md content"""
    return """
This is a sample demo using fasthtml. 

To run:

`pip install -r requirements.txt`

`python app.py`

"""

def generate_sample_fasthtml_content():
    """Generates boiler plate fasthtml content"""
    return """
# This was autogenerated as a sample application! 
from fh_bootstrap import *
app, rt = fast_app(pico=False, live=False)
        
@rt("/")
def get():
    return Titled(
            "Sample Demo",
            H1("This is a sample demo!")
            )
serve() 
"""

def generate_sample_dockerfile_content():
    """Generate boilerplate Dockerfile content."""
    return """
# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8000 available to the world outside this container
EXPOSE 8000

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]
"""

def generate_sample_cloudbuild_yaml_content():
    """Generate boilerplate cloudbuild.yaml content."""
    return """
steps:
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA', '.']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA']
# Deploy container image to Cloud Run
- name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
  entrypoint: gcloud
  args:
  - 'run'
  - 'deploy'
  - '$REPO_NAME'
  - '--image'
  - 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA'
  - '--region'
  - 'us-central1'
images:
- 'gcr.io/$PROJECT_ID/$REPO_NAME:$COMMIT_SHA'
"""

In [ ]:
#| export
@call_parse
def demo_garden_init(project_name: str = None, # this is the path for the directory you want demo garden to manage 
                    ):
    """Initializes a new Demo Garden project. Call this function when inside of the directory
    you want Demo Garden to manage. Using this like `git init`.
    """
    if project_name:
        project_dir = Path(project_name)
    else:
        project_dir = Path.cwd()
    if (project_dir / ".demogarden").exists():
        raise ValueError(f"Error: This is already a demogarden managed repository")
    if project_name and project_dir.exists():
        raise ValueError(f"Error: {project_name} already exists")

    (project_dir / "assets").mkdir(exist_ok=True)
    (project_dir / "demos").mkdir(exist_ok=True)
    (project_dir / "demos" / "Sample_Category" / "Demo_Name").mkdir(parents=True, exist_ok=True)
    with open(project_dir / "demos" / "Sample_Category" / "Demo_name" / "app.py", "w") as f:
        f.write(generate_sample_fasthtml_content())
    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "requirements.txt", "w") as f:
        f.write(generate_sample_requirements_txt())
    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "README.md", "w") as f:
        f.write(generate_sample_readme_content())

    # Generate Dockerfile for Demo Garden System. This is something that won't be configured by the user
    with open(project_dir / "Dockerfile", "w") as f:
        f.write(generate_sample_dockerfile_content())
    with open(project_dir / "cloudbuild.yaml", "w") as f:
        f.write(generate_sample_cloudbuild_yaml_content())
        
    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "Dockerfile", "w") as f:
        f.write(generate_sample_dockerfile_content())

    with open(project_dir / "demos" / "Sample_Category" / "Demo_Name" / "cloudbuild.yaml", "w") as f:
        f.write(generate_sample_cloudbuild_yaml_content())

    (project_dir / "app.py").touch()
    (project_dir / ".demogarden").touch()
    
    print(f"Demo Garden project initialized successfully in {project_dir}")

In [ ]:
#| export

def mk_landing_page():
    with open("landing.py", "w") as f:
        f.write("""from app import *

s1 = "Welcome to Demo Garden!"

def page():
    h2s = ['Welcome']
    txts = [Markdown(s1)]
    secs = Sections(h2s, txts)
    return BstPage(0, "Demo Garden Landing", *secs)
"""
        )


@call_parse
def demo_garden_build(
    path:str, # the folder demo garden will use to build `app.py`
                     ):
    """Build the Demo Garden project."""
   # if not Path(".demogarden").exists():
        #raise ValueError("Error: Not a demo garden project. Run 'demo_garden_init' to initialize this repository ")
    path = Path(path)
    cat_paths = [c for c in path.ls()] # these are the category paths
    cat_names = [c.name for c in path.ls()]
    demos_paths = [d.ls() for d in cat_paths] # these are the demo paths
    nav_items = [(name, f"/{name}") for name in cat_names]
    print(nav_items)
    mk_landing_page()
    
    with open("app.py", "w") as f:
        f.write(f"""
from fh_bootstrap import *
from itertools import chain
from markdown import markdown

md_exts='codehilite', 'smarty', 'extra', 'sane_lists'
def Markdown(s, exts=md_exts, **kw): return Div(NotStr(markdown(s, extensions=exts)), **kw)


def BstPage(selidx, title, *c):
    navitems = {nav_items}
    return (
        Title(title),
        Script('initTOC()'),
        Container(
            Navbar('nav', selidx, items=navitems, cls='navbar-light bg-secondary rounded-lg', placement=PlacementT.Default, expand=SizeT.Md, toggle_left=False),
            Toc(Container(H1(title, cls='pb-2 pt-1'), *c, cls='mt-3')),
        typ=ContainerT.Xl, cls='mt-3', data_bs_spy='scroll', data_bs_target='#toc'))

def Sections(h2s, texts):
    colors = 'yellow', 'pink', 'teal', 'blue'
    div_cls = 'py-2 px-3 mt-4 bg-light-{None} rounded-tl-lg'
    return chain([Div(H2(h2, id=f'sec{{i+1}}', cls=div_cls.format(colors[i%4])), Div(txt, cls='px-2'))
                  for i,(h2,txt) in enumerate(zip(h2s, texts))])
"""
        )
    
    for cat in cat_paths:
        with open(f"{cat.name}.py", "w") as f:
            f.write("from app import *")
            for demo in cat.ls():
                    f.write(f"""
def {demo.name}():
    with open("demos/{cat.name}/{demo.name}/app.py", "r") as f:
        contents = f.read()
    return contents
""")
            f.write(f"""
def page():
    caption = "Demos for {cat.name}"
    h2s = {[demo.name for demo in cat.ls()]}
    txts = {", ".join(f"Markdown({demo.name}())" for demo in cat.ls())}
    secs = Sections(h2s, txts)
    return BstPage({cat_names.index(cat.name) + 1}, "{cat.name}", *secs)
    
""")
    # Build app.py
    with open(f"main.py", "w") as f:
            f.write(f"""
from fh_bootstrap import *
import landing
import {", ".join(name for name in cat_names)}

hdrs = (
    Link(href='/assets/hl-styles.css', rel='stylesheet'),
    Link(href='/assets/styles.css', rel='stylesheet'),
)

app,rt = fast_app(pico=False, hdrs=bst_hdrs+hdrs, live=False)

app.get("/")(landing.page)

{"\n".join(
    f"app.get('/{name}')({name}.page)" for name in cat_names
)}

serve()
"""
            )
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()